In [10]:
import pyodbc
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-HMTBFS6\DB;'
                      'Database=ses;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()
pyodbc.drivers()


['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [ ]:
import matplotlib.pyplt as plt

In [11]:
cursor.execute('SELECT TOP(10) * FROM tablee')

In [13]:
import pandas as pd
data = pd.read_sql('SELECT TOP(10) * FROM tablee', conn)
data.head(10)

,SirketAdı,Panelsayısı,OrtkW,Ortkazanc,Komisyon,musterisayisi
0,Solarsan,4000,120000,80000,10,12
1,Gunessan,2000,60000,40000,10,5
2,assan,5000,140000,100000,8,18
3,wassan,1000,30000,20000,15,3
4,lapsan,6000,150000,120000,5,24
5,hapsan,3000,70000,50000,10,15


In [14]:
from sklearn.model_selection import train_test_split
y = data.OrtkW
X = data.drop(['OrtkW'], axis=1)

In [15]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [16]:
X_train.head(20)

,SirketAdı,Panelsayısı,Ortkazanc,Komisyon,musterisayisi
1,Gunessan,2000,40000,10,5
3,wassan,1000,20000,15,3
0,Solarsan,4000,80000,10,12
4,lapsan,6000,120000,5,24


In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [18]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [19]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 35300.0
